In [1]:
import numpy as np

## Initial setting

We create our two initial NumPy arrays in which we store our random sequence of 0 and 1 and the corresponding fitness.
We use numpy array because they are relatively small in size with respect to lists and we have homogeneous data.
In our implementation, with random fitness values, we generate (n_pop * n_bits + 1) random values to create the arrays.

To create a two-dimensional NumPy array, we use the method _coloumn_stack_ (it transform our two one-dimensional arrays into a two-dimensional one using a loop).

In [2]:
variability = 255
n_pop = 10
fitness = []
population = []
for i in range(n_pop):
    fitness.append(np.random.randint(0,1000))
    population.append([int(x) for x in '{:08b}'.format(np.random.randint(0,variability))])

fitness = np.array(fitness)
population = np.array(population)

# pop_array = np.column_stack([population, fitness])
# pop_array

## Fitness proportionate selection (or Roulette wheel selection)

In [13]:
def fit_prop_selection(pop, fitness):
    n_pop = len(pop)

    prob_fitness = fitness/sum(fitness)

    # Generate probability intervals for each individual
    probs = [sum(prob_fitness[:i+1]) for i in range(n_pop)]

    # Draw new population
    mating_pool = []
    for n in range(n_pop):
        r = np.random.uniform(0, 1)
        for (i, individual) in enumerate(pop):
            if probs[i] > r:
                mating_pool.append(individual)
                break

    return np.vstack(mating_pool)


In [14]:
fit_prop_selection(population, fitness)

array([[1, 1, 1, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0]])

In [15]:
timeit fit_prop_selection(population, fitness)

337 µs ± 94.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
timeit  [f/sum(fitness) for f in fitness] # 4 times slower

64.1 µs ± 8.64 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
timeit fitness/sum(fitness) 

14.4 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Linear Rank selection

In [18]:
def lin_rank_selection(pop, fitness):
    n = len(pop)
    
    sorted_ind = sorted(range(n), key=lambda k: fitness[k])
    fitness_rank = [2*pos/(n*(n+1)) for pos in range(1, n+1)]

    pop = pop[sorted_ind]

    # Draw new population
    mating_pool = []
    for j in range(n):
        s = 0
        r = np.random.uniform(0, 1)
        for (i, individual) in enumerate(pop):
            s += fitness_rank[i]
            if  s > r:
                mating_pool.append(individual)
                break

    return np.vstack(mating_pool)

In [19]:
lin_rank_selection(population, fitness)

array([[0, 1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 1, 1]])

In [20]:
timeit lin_rank_selection(population, fitness)

342 µs ± 97.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Tournament selection

In [21]:
def tournament_selection(pop, fitness, k=3):
    n_pop = len(pop)
    mating_pool = []

    for i in range(n_pop):
        best = np.random.randint(n_pop)

        for j in range(k):
            selected = np.random.randint(n_pop)
            if fitness[selected] > fitness[best]:
                    best = selected
        
        mating_pool.append(pop[best])

    return np.vstack(mating_pool)


In [22]:
tournament_selection(population, fitness)

array([[0, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0]])

In [23]:
timeit tournament_selection(population, fitness)

743 µs ± 269 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Truncation selection

In [24]:
def truncation_selection(pop, fitness, trunc_perc = 60):
    
    sorted_ind = sorted(range(len(fitness)), key=lambda k: fitness[k], reverse = True)

    num = len(pop)*trunc_perc//100

    mating_pool = pop[sorted_ind][:num]      

    return mating_pool


In [25]:
truncation_selection(population, fitness)

array([[0, 0, 1, 0, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 0]])

In [26]:
timeit truncation_selection(population, fitness)

32.6 µs ± 8.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
